In [1]:
import sys
sys.path.append("../../")

In [3]:
from cad.calc.parameters import NazareShape
from cad.calc.loss import LossFunction
import numpy as np
from cad.calc.conv import note_to_freq
from cad.calc.geo import Geo

In [19]:
class SintraLoss(LossFunction):
        
    def __init__(self):
        LossFunction.__init__(self)
        
        base_note = -31
        self.target_notes = base_note + np.concatenate([np.array((0,4,7)) + 12*n for n in range(0,5)])
        self.target_freqs = np.log2(note_to_freq(self.target_notes))
        #self.multiples = np.arange(1,15)*note_to_freq(base_note)

    def get_brightness(self, geo):
        imp = geo.get_cadsd().get_impedance_spektrum()
        return (imp.query("freq>=400")).impedance.sum() / imp.impedance.sum()

    def get_deviations(self, freq, reference):
        
        deviations = []
        for f in freq:
            d = [np.abs(r-f) for r in reference]
            deviations.append(np.min(d))
        return deviations
        
    def get_loss(self, geo, context=None):
        
        notes = geo.get_cadsd().get_notes()
        freqs = np.log2(list(notes.freq))
        toots = freqs
        
        deviations = self.get_deviations(toots, self.target_freqs)
        fundamental_loss = deviations[0]
        fundamental_loss *= 3
        toots_loss = np.sum(deviations[1:] / np.arange(1, len(deviations[1:])+1))
        toots_loss *= 1

        brightness_loss = self.get_brightness(geo)
        brightness_loss *= 1
        
        loss = {
            "loss": fundamental_loss + toots_loss + brightness_loss,
            "fundamental_loss": fundamental_loss,
            "toots_loss": toots_loss,
            "brightness_loss": brightness_loss
        }
        return loss
    
loss = SintraLoss()
shape = NazareShape()
loss(shape.make_geo())

{'loss': 0.7571635113379838,
 'fundamental_loss': 0.12113507740126295,
 'toots_loss': 0.22173074400440385,
 'brightness_loss': 0.41429768993231697}

In [13]:
np.ones(5) * np.arange(1,6)

array([1., 2., 3., 4., 5.])